In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# model imports
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier as KNN

import xgboost as xgb

# imports from starter code by SRK
import os
import sys
import operator
from scipy import sparse
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

### Import data

In [2]:
data_path = "../data/"
train_file = data_path + "churn_test.csv"
test_file = data_path + "churn_train.csv"
train_df = pd.read_csv(train_file, parse_dates=['last_trip_date', 'signup_date'], infer_datetime_format=True)
test_df = pd.read_csv(test_file, parse_dates=['last_trip_date', 'signup_date'], infer_datetime_format=True)

today =pd.to_datetime('2014-07-01')
train_df['days_since_trip'] =  (today - train_df['last_trip_date']).dt.days
test_df['days_since_trip'] =  (today - test_df['last_trip_date']).dt.days

y_train = (train_df['days_since_trip'].values > 30) * 1
y_test = (test_df['days_since_trip'].values > 30) * 1

# Let us extract some features like year, month, day, hour from date columns #
train_df["signup_year"] = train_df["signup_date"].dt.year
test_df["signup_year"] = test_df["signup_date"].dt.year
train_df["signup_month"] = train_df["signup_date"].dt.month
test_df["signup_month"] = test_df["signup_date"].dt.month
train_df["signup_day"] = train_df["signup_date"].dt.day
test_df["signup_day"] = test_df["signup_date"].dt.day
train_df["signup_day_of_week"] = train_df["signup_date"].dt.dayofweek
test_df["signup_day_of_week"] = test_df["signup_date"].dt.dayofweek

train_df.drop(['last_trip_date','days_since_trip','signup_date'], axis=1, inplace=True)
test_df.drop(['last_trip_date','days_since_trip','signup_date'], axis=1, inplace=True)

print(train_df.shape)
print(test_df.shape)

(10000, 14)
(40000, 14)


In [3]:
y_test.min()

0

### Which columns have nulls?

In [4]:
has_nulls = []
for column in train_df.columns:
    if train_df[column].isnull().sum() > 0:
        has_nulls.append(column)
        print("column {} has {} nulls".format(column, train_df[column].isnull().sum()))
        print(train_df[column].describe())
        print("")

column avg_rating_by_driver has 39 nulls
count    9961.000000
mean        4.781056
std         0.440871
min         1.000000
25%              NaN
50%              NaN
75%              NaN
max         5.000000
Name: avg_rating_by_driver, dtype: float64

column avg_rating_of_driver has 1594 nulls
count    8406.000000
mean        4.601011
std         0.627343
min         1.000000
25%              NaN
50%              NaN
75%              NaN
max         5.000000
Name: avg_rating_of_driver, dtype: float64

column phone has 77 nulls
count       9923
unique         2
top       iPhone
freq        6954
Name: phone, dtype: object



/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [5]:
has_nulls = []
for column in test_df.columns:
    if test_df[column].isnull().sum() > 0:
        has_nulls.append(column)
        print("column {} has {} nulls".format(column, test_df[column].isnull().sum()))
        print(test_df[column].describe())
        print("")
has_nulls

column avg_rating_by_driver has 162 nulls
count    39838.000000
mean         4.777434
std          0.448088
min          1.000000
25%               NaN
50%               NaN
75%               NaN
max          5.000000
Name: avg_rating_by_driver, dtype: float64

column avg_rating_of_driver has 6528 nulls
count    33472.000000
mean         4.601697
std          0.614810
min          1.000000
25%               NaN
50%               NaN
75%               NaN
max          5.000000
Name: avg_rating_of_driver, dtype: float64

column phone has 319 nulls
count      39681
unique         2
top       iPhone
freq       27628
Name: phone, dtype: object



/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


['avg_rating_by_driver', 'avg_rating_of_driver', 'phone']

In [6]:
# Replacing nulls with means and adding their null status as a column
for column in ['avg_rating_by_driver', 'avg_rating_of_driver']:
    train_df[column+'_null'] = train_df[column].isnull()*1
    train_df[column] = train_df[column].fillna(train_df[column].mean())
    test_df[column+'_null'] = test_df[column].isnull()*1
    test_df[column] = test_df[column].fillna(test_df[column].mean())

# preserving the columns with missing phone info
train_df['phone'] = train_df['phone'].fillna('Other')
test_df['phone'] = test_df['phone'].fillna('Other')

# changing to int 
train_df['luxury_car_user'] = train_df['luxury_car_user']*1
test_df['luxury_car_user'] = test_df['luxury_car_user']*1

In [7]:
train_df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,signup_year,signup_month,signup_day,signup_day_of_week,avg_rating_by_driver_null,avg_rating_of_driver_null
0,2.48,5.0,5.000000,1.0,Winterfell,Android,0.0,2,1,100.0,2014,1,6,0,0,0
1,10.81,5.0,5.000000,1.0,Winterfell,iPhone,0.0,3,1,100.0,2014,1,6,0,0,0
2,12.95,5.0,5.000000,1.0,Astapor,Android,0.0,1,1,100.0,2014,1,19,6,0,0
3,3.92,5.0,4.601011,1.0,Winterfell,iPhone,0.0,0,0,0.0,2014,1,9,3,0,1
4,1.46,5.0,4.500000,1.0,Astapor,iPhone,0.0,2,0,100.0,2014,1,7,1,0,0


In [8]:
# Changing categorical to dummies
obj_cols = [column for column in train_df.columns if train_df[column].dtype == 'O']

for column in obj_cols:
    dummies_df = pd.get_dummies(train_df[column])
    train_df = pd.concat([train_df, dummies_df], axis=1)
    dummies_df = pd.get_dummies(train_df[column])
    test_df = pd.concat([test_df, dummies_df], axis=1)

In [9]:
train_df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,...,signup_day,signup_day_of_week,avg_rating_by_driver_null,avg_rating_of_driver_null,Astapor,King's Landing,Winterfell,Android,Other,iPhone
0,2.48,5.0,5.000000,1.0,Winterfell,Android,0.0,2,1,100.0,...,6,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0
1,10.81,5.0,5.000000,1.0,Winterfell,iPhone,0.0,3,1,100.0,...,6,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0
2,12.95,5.0,5.000000,1.0,Astapor,Android,0.0,1,1,100.0,...,19,6,0,0,1.0,0.0,0.0,1.0,0.0,0.0
3,3.92,5.0,4.601011,1.0,Winterfell,iPhone,0.0,0,0,0.0,...,9,3,0,1,0.0,0.0,1.0,0.0,0.0,1.0
4,1.46,5.0,4.500000,1.0,Astapor,iPhone,0.0,2,0,100.0,...,7,1,0,0,1.0,0.0,0.0,0.0,0.0,1.0


In [10]:
train_df.drop(['city','phone'], axis=1, inplace=True)
test_df.drop(['city','phone'], axis=1, inplace=True)

In [11]:
test_df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,signup_year,signup_month,signup_day,signup_day_of_week,avg_rating_by_driver_null,avg_rating_of_driver_null,Astapor,King's Landing,Winterfell,Android,Other,iPhone
0,6.94,5.0,5.000000,1.00,0.0,0,0,100.0,2014,1,12,6,0,0,0.0,0.0,1.0,1.0,0.0,0.0
1,8.06,5.0,5.000000,1.00,0.0,2,1,0.0,2014,1,25,5,0,0,0.0,0.0,1.0,0.0,0.0,1.0
2,21.50,4.0,4.601697,1.00,0.0,1,1,100.0,2014,1,2,3,0,1,1.0,0.0,0.0,1.0,0.0,0.0
3,9.46,5.0,4.601697,2.75,100.0,1,0,100.0,2014,1,9,3,0,1,0.0,0.0,1.0,0.0,0.0,1.0
4,13.77,5.0,4.601697,1.00,0.0,0,0,100.0,2014,1,31,4,0,1,1.0,0.0,0.0,0.0,0.0,1.0


In [12]:
train_df.describe()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,signup_year,signup_month,signup_day,signup_day_of_week,avg_rating_by_driver_null,avg_rating_of_driver_null,Astapor,King's Landing,Winterfell,Android,Other,iPhone
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.0,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.818925,4.781056,4.601011,1.073996,8.81831,2.268200,0.380800,61.132890,2014.0,1.0,16.565400,3.304300,0.003900,0.159400,0.329800,0.198600,0.471600,0.296900,0.007700,0.695400
std,5.704789,0.440010,0.575169,0.221982,19.73744,3.717506,0.485608,37.050156,0.0,0.0,8.773438,1.884372,0.062331,0.366067,0.470164,0.398966,0.499218,0.456915,0.087416,0.460261
min,0.000000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,2014.0,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.420000,4.700000,4.500000,1.000000,0.00000,0.000000,0.000000,33.300000,2014.0,1.0,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.890000,5.000000,4.800000,1.000000,0.00000,1.000000,0.000000,66.700000,2014.0,1.0,17.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,6.990000,5.000000,5.000000,1.050000,9.10000,3.000000,1.000000,100.000000,2014.0,1.0,24.000000,5.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
max,79.340000,5.000000,5.000000,5.000000,100.00000,71.000000,1.000000,100.000000,2014.0,1.0,31.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Cross Validation

In [13]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 2
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['nthread'] = 4
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20, verbose_eval=False)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [14]:
# ymin = min(y_train.min(),y_test.min())
# ymax = max(y_train.max(),y_test.max())
# y_train = (y_train - ymin) / float(ymax)
# y_test = (y_test - ymin) / float(ymax)

In [15]:
X_train = train_df.values
X_test = test_df.values

cv_scores_xgb = []
kf = model_selection.KFold(n_splits=10, random_state=1)

#dev is the current fold's train, and val is current fold's validation
for dev_index, val_index in kf.split(range(X_train.shape[0])):    # excluded since we are not stratefying ,train_y):
        print('.', end='', flush=True)
        dev_X, val_X = X_train[dev_index,:], X_train[val_index,:]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
#         print(val_y.min(),val_y.max(), preds.min(), preds.max())
        cv_scores_xgb.append([log_loss(val_y, preds), accuracy_score(val_y, (preds[:,1] > 0.5)*1)])
cv_scores_xgb = np.array(cv_scores_xgb)
print("")
print("XGB mean score logloss {}, accuracy {}".format(np.mean(cv_scores_xgb[:,0]), np.mean(cv_scores_xgb[:,1])))

..........
XGB mean score logloss 0.46404540859926496, accuracy 0.7762


In [18]:
preds

array([[ 0.15114838,  0.84885162],
       [ 0.26051676,  0.7394833 ],
       [ 0.22066559,  0.77933437],
       ..., 
       [ 0.55183148,  0.44816846],
       [ 0.02620532,  0.97379464],
       [ 0.28798062,  0.71201938]], dtype=float32)